In [20]:
import torch
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

import numpy as np
from tqdm import tqdm

from resnet import ResNet18

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.cuda.is_available()

True

In [38]:
CIFAR10_TRAIN_MEAN = (0.49139968, 0.48215827, 0.44653124)
CIFAR10_TRAIN_STD = (0.24703233, 0.24348505, 0.26158768)

BATCH_SIZE = 1

mean = CIFAR10_TRAIN_MEAN
std = CIFAR10_TRAIN_STD

train_transformer = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize(mean, std)
])

test_transformer = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize(mean, std)
])

train_ds = CIFAR10(root='../datasets', train=True, download=True, transform=train_transformer)
test_ds = CIFAR10(root='../datasets', train=False, download=True, transform=test_transformer)
LEN_TRAIN = len(train_ds)
LEN_TEST = len(test_ds)
classes = 10

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [39]:
model = ResNet18(classes = classes).to(device)
model.to(device)
model.load_state_dict(torch.load('../models/epoch39_train0.98982_test0.9154.pt'))

<All keys matched successfully>

In [40]:
for param in model.parameters():
    param.requires_grad = False

In [41]:
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

11,173,962 total parameters.
0 training parameters.


In [42]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [47]:
test_acc = 0
with tqdm(train_loader, unit="batch") as tepoch:
    for xtrain, ytrain in tepoch:       
        xtrain = xtrain.to(device)
        
        model.bn1.register_forward_hook(get_activation('bn1'))
        output = model(xtrain)
        features = activation['bn1'].cpu().detach().numpy()
        
        test_pred = torch.max(output.cpu(),1).indices
        test_acc += int(torch.sum(test_pred == ytrain))

        print(test_acc)
        break

  0%|          | 0/50000 [00:00<?, ?batch/s]

1
